In [17]:
import google.generativeai as genai
from google.colab import userdata
api_key = userdata.get('GEMINI_API_KEY')
genai.configure(api_key=api_key)
# Agora O Api do Gemini irá funcionar sem erro.
model = genai.GenerativeModel('gemini-2.5-flash')


In [11]:
import pandas as pd
import json

In [15]:
# Passo 0: Criar o arquivo CSV conforme a condição do problema
data = {'UserID': [1, 2, 3, 4, 5]}
df_ids = pd.DataFrame(data)
df_ids.to_csv('SDW2023.csv', index=False)

In [13]:
# Criando o Banco de dados que a API forneceria.
users_db = [
    {"id": 1, "name": "Arthur", "news": []},
    {"id": 2, "name": "Bruno", "news": []},
    {"id": 3, "name": "Ana", "news": []},
    {"id": 4, "name": "Maria", "news": []},
    {"id": 5, "name": "João", "news": []}
]

In [19]:
#Extração
ids_from_csv = pd.read_csv('SDW2023.csv')['UserID'].tolist()

def extract_user_data(user_id, database):
    return next((user for user in database if user['id'] == user_id), None)


users = [extract_user_data(id, users_db) for id in ids_from_csv if extract_user_data(id, users_db) is not None]
print(f"Extraídos {len(users)} usuários com sucesso.")

Extraídos 5 usuários com sucesso.


In [ ]:
#Transforamção.

In [20]:
def generate_ai_news(user):
    """Gera uma mensagem personalizada usando o Gemini"""
    # Criamos um prompt contextualizado com o nome do usuário
    prompt = f"Crie uma dica de investimento personalizada para {user['name']}. Seja breve e use no máximo 10 palavras."

    try:
        response = model.generate_content(prompt)
        return response.text.strip()
    except Exception as e:
        return f"Erro ao gerar mensagem: {e}"

In [21]:
for user in users:
    print(f"Processando usuário: {user['name']}...")
    news_content = generate_ai_news(user)

    # Adicionando a mensagem no formato esperado pela estrutura original da API
    user['news'].append({
        "description": news_content
    })

print("\nTransformação concluída com sucesso!")

Processando usuário: Arthur...
Processando usuário: Bruno...
Processando usuário: Ana...
Processando usuário: Maria...
Processando usuário: João...

Transformação concluída com sucesso!


In [24]:
for user in users:
    # Acessamos o nome do usuário
    nome = user['name']

    # Acessamos a última mensagem adicionada na lista 'news'
    # Pegamos o campo 'description' que contém o texto do Gemini
    mensagem = user['news'][-1]['description']

    print(f"Usuário: {nome}")
    print(f"Mensagem: {mensagem}")
    print("-" * 30)

Usuário: Arthur
Mensagem: Arthur, diversifique sempre seus investimentos.
------------------------------
Usuário: Bruno
Mensagem: Bruno, diversifique com cautela para um futuro seguro.
------------------------------
Usuário: Ana
Mensagem: Ana, diversifique e invista com visão de longo prazo.
------------------------------
Usuário: Maria
Mensagem: Maria, invista em você. Conheça e aproveite seus talentos!
------------------------------
Usuário: João
Mensagem: João, defina seus objetivos e invista com disciplina e foco.
------------------------------


In [25]:
import json

# Salvando a lista final de usuários com as mensagens da IA em um arquivo JSON
output_file = 'SDW2023_final.json'

with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(users, f, ensure_ascii=False, indent=4)

print(f"Arquivo '{output_file}' gerado com sucesso! O pipeline ETL foi finalizado.")

Arquivo 'SDW2023_final.json' gerado com sucesso! O pipeline ETL foi finalizado.
